In [1]:
import xml.etree.ElementTree as ET

In [17]:
Comments_tree = ET.parse('Data_Dump/Comments.xml')
Posts_tree = ET.parse('Data_Dump/Posts.xml')
Users_tree = ET.parse('Data_Dump/Users.xml')
Tags_tree = ET.parse('Data_Dump/Tags.xml')

In [20]:
Comments_root = Comments_tree.getroot()
Posts_root = Posts_tree.getroot()
Users_root = Users_tree.getroot()
Tags_root = Tags_tree.getroot()

In [21]:
for child in Comments_root[:5]:
    print(child.attrib)

{'Id': '3', 'PostId': '3', 'Score': '2', 'Text': "Good naming convention and well structured code will help you decrease the comments need. Don`t forget that each line of comments you add it's a new line to maintain!!", 'CreationDate': '2010-09-01T19:47:32.873', 'UserId': '28', 'ContentLicense': 'CC BY-SA 2.5'}
{'Id': '5', 'PostId': '20', 'Score': '17', 'Text': '+1 for truth, -1 for practicality. :)', 'CreationDate': '2010-09-01T19:49:47.933', 'UserId': '9', 'ContentLicense': 'CC BY-SA 2.5'}
{'Id': '8', 'PostId': '3', 'Score': '0', 'Text': '@Gabriel: that was already in my answer, look at the end.', 'CreationDate': '2010-09-01T19:51:45.260', 'UserId': '11', 'ContentLicense': 'CC BY-SA 2.5'}
{'Id': '9', 'PostId': '23', 'Score': '2', 'Text': 'Sometimes it\'s the technique I use, but also when I\'m too deep into my work and I want to change my mind to get a "fresh" look to my work.', 'CreationDate': '2010-09-01T19:51:48.200', 'UserId': '28', 'ContentLicense': 'CC BY-SA 2.5'}
{'Id': '16', 

In [22]:
len(Tags_root)

1660

In [23]:
len(Users_root)

316020

In [24]:
len(Posts_root)

224241